<a href="https://colab.research.google.com/github/trandangtrungduc/ChatBot/blob/main/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **TABLE OF CONTENTS**

---


## 1. Bot based on Gutenberg data 

## 2. Bot based on available data




---


## **1. Bot based on Gutenberg data** 




### 1.1 Import necessary librabries and Jane Austen's novels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import re
import nltk
import requests
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

nltk.download('punkt')

In [3]:
Novels = [
          {'author': 'Jane Austen - Persuasion', 'url': 'https://www.gutenberg.org/files/105/105-8.txt'},
          {'author': 'Jane Austen - Lady Susan', 'url': 'https://www.gutenberg.org/files/946/946.txt'},
          {'author': 'Jane Austen - Emma', 'url': 'https://www.gutenberg.org/files/158/158-0.txt'}
         ]
         
PATH = '/content/drive/MyDrive/Data/Chat_Bot/Data1'
PATH_AT = '/content/drive/MyDrive/Data/Chat_Bot/NewData1'
start_point = ['***', 'START', 'OF', 'PROJECT','GUTENBERG',]
end_point = ['END', 'PROJECT', 'GUTENBERG',]

i = 0
for novel in Novels:
  # Get content
  content = requests.get(novel['url']).text
  # Define the folder
  folder = PATH + f'/{novel["author"]}/'
  if not os.path.exists(folder):
      os.makedirs(folder)
  # Save the text
  with open(folder + f'novel{i}.txt', 'wb') as f:
      f.write(str(content).encode())
  i += 1

In [4]:
# File location
clean_sentence = ''
DOC_PATTERN = r'.*\.txt'
novel_corpus = PlaintextCorpusReader(PATH, DOC_PATTERN)
def cleaner(text, limit=0):
    # Regex pattern for only alphanumeric, hyphenated text with 4 or more chars
    pattern = re.compile(r"[A-Za-z0-9\-]{5,60}")
    return re.findall(pattern, text)

i = 0
for fileid in novel_corpus.fileids():
  paras_of_fileid = novel_corpus.paras(fileid)
  author = fileid.split('/')[0]
  for para in paras_of_fileid:
    bigsent = [w.upper() for w in para[0]]  
    if set(start_point).issubset(para[0]):
      startpoint = paras_of_fileid.index(para)+1
    if set(end_point).issubset(bigsent):
      endpoint = paras_of_fileid.index(para)

  # Store this in an authortext folder for analysis on authors
  for para in paras_of_fileid[startpoint:endpoint]:
    for sent in para:
      # Clean the sentences
      clean_sentence += ' '.join(cleaner(' '.join(sent))) + ' '
    clean_sentence += '\n'  
      
  # Open and save the text to a file
  folder = PATH_AT + f'/{author}/'
  if not os.path.exists(folder):
      os.makedirs(folder)
  # Save the text
  with open(folder + f'cleaned_novel.txt', 'a+') as f:
      f.write(clean_sentence)
  clean_sentence = '' # Reset the sentences

In [5]:
# Corpus statistics
new_novel_corpus = PlaintextCorpusReader(PATH_AT, DOC_PATTERN)
# Dataframe of sentences and authors
for fileid in new_novel_corpus.fileids():
  novel = pd.DataFrame(new_novel_corpus.raw(fileids=fileid).split('\n'), columns=['sentence'])
  novel['author'] = fileid.split('/')[0]
  try:
    novels = pd.concat([novels, novel])
  except NameError: 
    novels = novel
novel_1 = novel.sentence.to_list()
novel_1[6:10]

['Walter Elliot Kellynch Somersetshire amusement never Baronetage there found occupation consolation distressed there faculties roused admiration respect contemplating limited remnant earliest patents there unwelcome sensations arising domestic affairs changed naturally contempt turned almost endless creations century there every other powerless could history interest which never failed which favourite volume always opened ',
 'ELLIOT KELLYNCH ',
 'Walter Elliot March married Elizabeth daughter James Stevenson South county Gloucester which issue Elizabeth August still November November ',
 'Precisely paragraph originally stood printer hands Walter improved adding information himself family these words after birth Married December Charles Charles Musgrove Uppercross county Somerset inserting accurately month which ']

In [6]:
def TFIDF(novel):
  TfidfVec = TfidfVectorizer(stop_words='english')
  tfidf = TfidfVec.fit_transform(novel)
  X_TfidVec = pd.DataFrame(tfidf.toarray(), columns=TfidfVec.get_feature_names(), dtype='float32')
  return TfidfVec, X_TfidVec
def BOW(novel):
  BowVec = CountVectorizer(stop_words='english')
  Bow = BowVec.fit_transform(novel)
  X_BowVec = pd.DataFrame(Bow.toarray(), columns=BowVec.get_feature_names(), dtype='float32')
  return BowVec, X_BowVec

In [7]:
TfidfVec_1, X_TfidVec_1 = TFIDF(novel_1)
BowVec_1, X_BowVec_1 = BOW(novel_1)
X_TfidVec_1.head(3)

,abbreviation,abdication,abide,abilities,abode,abominable,abominate,abominates,abroad,absence,absent,absenting,absolute,absolutely,abstraction,absurd,absurdity,abundance,abuse,abused,abydos,accent,accept,acceptable,acceptance,accepted,accepting,accession,accessions,accident,accidental,accidentally,accommodate,accommodating,accommodation,accommodations,accompanied,accompany,accompanying,accomplish,...,world,worldly,worse,worst,worsting,worth,worthy,wound,wounded,wrapt,wreck,wretched,wretchedly,wretchedness,wriggles,wrinkles,wrist,write,writing,written,wrong,wrote,wrought,yarmouth,yearly,years,yeomen,yesterday,yestermorn,yield,yielded,yielding,young,younger,youngest,younker,youth,youthful,zealous,zealously
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X_BowVec_1.head(3)

,abbreviation,abdication,abide,abilities,abode,abominable,abominate,abominates,abroad,absence,absent,absenting,absolute,absolutely,abstraction,absurd,absurdity,abundance,abuse,abused,abydos,accent,accept,acceptable,acceptance,accepted,accepting,accession,accessions,accident,accidental,accidentally,accommodate,accommodating,accommodation,accommodations,accompanied,accompany,accompanying,accomplish,...,world,worldly,worse,worst,worsting,worth,worthy,wound,wounded,wrapt,wreck,wretched,wretchedly,wretchedness,wriggles,wrinkles,wrist,write,writing,written,wrong,wrote,wrought,yarmouth,yearly,years,yeomen,yesterday,yestermorn,yield,yielded,yielding,young,younger,youngest,younker,youth,youthful,zealous,zealously
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0




---



In [9]:
PATH = '/content/drive/MyDrive/Data/Chat_Bot/Data2/'
def clean_text(text):
    with open(novel_2, 'r', encoding='utf8', errors='ignore') as f:
        # Lowercase the novel
        text = f.read().lower()  
        # Replace markdown characters
        text = text.replace('\xa0', ' ')
        # Remove emoji and author name
        text = re.sub(r"✍ jane austen", "", text)      
        # Remove these weird chars
        text = text.replace('—', ' ')
        text = text.replace('_', ' ')
        text = text.replace("£", " ")
        text = re.sub(r"\n", " ", text)
        # Remove numbers like 1st, 2nd, 3rd...
        text = re.sub(r"\d+\w*", " ", text)
        # Remove volume, chapter, headings and title
        text = re.sub(r"#+\s\w+\s\w+\s", "", text)
        text = re.sub(r"#\s\w+\s", "", text)  
    return text
# Clean the text    
cleaned_text = ''
for file in os.listdir(PATH):
    novel_2 = PATH + file
    cleaned_text += ' ' + clean_text(novel_2)

In [ ]:
cleaned_text[0:500]

'    #   sir walter elliot, of kellynch hall, in somersetshire, was a man who, for his own amusement, never took up any book but the baronetage; there he found occupation for an idle hour, and consolation in a distressed one; there his faculties were roused into admiration and respect, by contemplating the limited remnant of the earliest patents; there any unwelcome sensations, arising from domestic affairs changed naturally into pity and contempt as he turned over the almost endless creations of'

In [10]:
sent_tokens = nltk.sent_tokenize(cleaned_text)
# Function support extract sentence from novel
def Extract_Sentence(text):
    sentence_pattern = '(?<=“)(.+?)(?=”)'
    m = re.search(sentence_pattern, text)
    if m is not None:
        return m.group(0)
# List of sentence of novel
sentence = [Extract_Sentence(sentence) for sentence in sent_tokens if Extract_Sentence(sentence)]
# Apply TFIDF and Bag of Words
TfidfVec_2, X_TfidVec_2 = TFIDF(sentence)
BowVec_2, X_BowVec_2 = BOW(sentence)

In [ ]:
X_BowVec_2.head(3)

,abbey,able,abominable,abroad,absent,absolutely,accident,account,accounts,acknowledged,acquaintance,acquainted,actually,added,admiral,admiralty,admire,admired,admissible,advantage,advertise,advice,advise,affection,afraid,ago,agree,agreeable,agreeableness,agreed,ah,aimed,air,alarmed,alderneys,alike,allow,allowed,allowing,altered,...,weymouth,whim,whims,wife,william,willing,willingness,wilson,winds,winthrop,wish,wished,wishes,wishing,wit,woman,women,wonder,wonderful,woodhouse,word,working,world,worse,worst,worth,worthy,wounding,wretched,write,writes,writing,wrong,ye,year,years,yes,yesterday,young,youngest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_TfidVec_2.head(3)

,abbey,able,abominable,abroad,absent,absolutely,accident,account,accounts,acknowledged,acquaintance,acquainted,actually,added,admiral,admiralty,admire,admired,admissible,advantage,advertise,advice,advise,affection,afraid,ago,agree,agreeable,agreeableness,agreed,ah,aimed,air,alarmed,alderneys,alike,allow,allowed,allowing,altered,...,weymouth,whim,whims,wife,william,willing,willingness,wilson,winds,winthrop,wish,wished,wishes,wishing,wit,woman,women,wonder,wonderful,woodhouse,word,working,world,worse,worst,worth,worthy,wounding,wretched,write,writes,writing,wrong,ye,year,years,yes,yesterday,young,youngest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Function define bot type
def select_bot(number):
    switcher_1 = {1:TfidfVec_1, 2:BowVec_1, 3:TfidfVec_2, 4:BowVec_2}
    return switcher_1.get(number)
# Answer vector to calculate the similarity of question and answer 
def vec_answer(number):
    switcher_2 = {1:X_TfidVec_1, 2:X_BowVec_1, 3:X_TfidVec_2, 4:X_BowVec_2}
    return switcher_2.get(number)
# Resource to extract the answer
def source_answer(number):
    switcher_3 = {1:novel_1, 2:novel_1, 3:sentence, 4:sentence}
    return switcher_3.get(number)
# Similarity function
def consine_dis(x,y):
    cosine_simmilarity = (x@y.T)/(np.linalg.norm(x.toarray())*np.linalg.norm(y))
    return cosine_simmilarity
def euclidean_dis(x,y):
    euclid_similarity = np.sqrt(np.sum(np.power(x-y,2),axis=1))
    return euclid_similarity 
# Function define greetings
greetings = ['good morning', 'good afternoon', 'hi', 'hello', 'how are you?']
greetings_answer = ['Good morning!', 'Good afternoon!', 'Hi!', 'Hello!', 'I\'m good, thanks!']

In [14]:
# Function support answer for user
def answer(user_answer, bot):
    # Preprocess for input
    bot_answer = ''
    question = select_bot(bot).transform([user_answer])
    consine_similarity = consine_dis(question,vec_answer(bot))
    euclid_similarity = euclidean_dis(question.toarray(),vec_answer(bot).to_numpy())

    # Couldn't find a sentence close to the user's question
    #if consine_similarity.argmax() == 0:
        #return "I'm sorry? I don't get what you mean."
    # Extract the answer
    bot_answer = source_answer(bot)[euclid_similarity.argmin()] 
    bot_answer = bot_answer[0:-1] + '.'  
    # Capitalize the first letter of the answer 
    return bot_answer.capitalize()
def conversation():
  # Loop to select bot between 1 and 4
  flag_1 = True
  print("Please choose from 1 to 4 because of the limit on the number of bots.")
  while (flag_1 == True):
    try:
      bot = int(input())
      if bot in [1,2,3,4]:
        flag_1 = False
    except:
      print("Please choose from 1 to 4 because of the limit on the number of bots.")
  # Opening sentence
  print("BOT: Hi, this is Bot " + str(bot) + "." + " I will try to respond to you resonably. If you want to exit, please type \'Bye\'.") 
  # Loop to communicate with user
  flag_2 = True
  while flag_2 == True:
      # Handle input
      user_answer = input()
      user_answer = user_answer.lower()    
      # Respond from bot
      if user_answer == 'bye':
          flag_2 = False
          print("BOT: You are welcome. Goodbye!")
      elif user_answer in greetings:
          print(greetings_answer[greetings.index(user_answer)])
      else:
          print("BOT: ", end="")
          print(answer(user_answer, bot))

In [15]:
conversation()

Please choose from 1 to 4 because of the limit on the number of bots.
3
BOT: Hi, this is Bot 3. I will try to respond to you resonably. If you want to exit, please type 'Bye'.
hi
Hi!
my name is elliot
BOT: Elliot.
let me see your hand
BOT: Let me entreat you.
i need some drinks tonight


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


BOT: As to all that.
bye
BOT: You are welcome. Goodbye!
